<a href="https://colab.research.google.com/github/MBilalSharif/RAG-based-CV-Reader/blob/main/CV_Reader_RAG_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [34]:
!pip install -q langchain langchain-community chromadb sentence-transformers pypdf google-generativeai
print("Installed!")

Installed!


In [35]:
# Directly use your CV file
cv_filename = "M.Bilal Sharif.pdf"
print(f"✅ Using CV file: {cv_filename}")

✅ Using CV file: M.Bilal Sharif.pdf


In [36]:
from langchain_community.document_loaders import PyPDFLoader, TextLoader
from langchain_core.documents import Document

if cv_filename.endswith('.pdf'):
    loader = PyPDFLoader(cv_filename)
    documents = loader.load()
elif cv_filename.endswith('.txt'):
    loader = TextLoader(cv_filename)
    documents = loader.load()
else:
    with open(cv_filename, 'r') as f:
        documents = [Document(page_content=f.read())]

print(f"Loaded {len(documents)} document(s)")
print(f"Total characters: {sum(len(doc.page_content) for doc in documents)}")

Loaded 1 document(s)
Total characters: 5170


In [37]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

#Chunking doc
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=200,
    chunk_overlap=100
)

chunks = text_splitter.split_documents(documents)
print(f"Created {len(chunks)} chunks")

Created 41 chunks


In [38]:
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma

# Creating embeddings
embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)

print("Creating vector database...")
vectorstore = Chroma.from_documents(
    documents=chunks,
    embedding=embeddings,
    persist_directory="./cv_vectorstore"
)
print("Vector database created!")

Loading weights:   0%|          | 0/103 [00:00<?, ?it/s]

BertModel LOAD REPORT from: sentence-transformers/all-MiniLM-L6-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


Creating vector database...
Vector database created!


In [40]:
import google.generativeai as genai
from getpass import getpass


api_key = getpass("🔑 Paste your Google AI API key: ")
genai.configure(api_key=api_key)

model = genai.GenerativeModel(
    model_name='models/gemini-2.5-flash',
    generation_config={
        'temperature': 0.3,
        'max_output_tokens': 500,
    }
)

print("Gemini ready!")

🔑 Paste your Google AI API key: ··········
Gemini ready!


In [46]:
def ask_cv_question(question, k=5):
    """Ask a question about your CV using RAG"""


    print(f"🔍 Searching for relevant info...")
    relevant_docs = vectorstore.similarity_search(question, k=k)
    context = "\n\n".join([doc.page_content for doc in relevant_docs])
    print(f"✅ Found {len(relevant_docs)} relevant sections\n")


    prompt = f"""You must respond with ONLY the answer, nothing else.

CV Information:
{context}

Question: {question}

Rules:
1. Extract the exact answer from the CV
2. Do NOT add any preamble like "The answer is" or "According to"
3. Do NOT add explanations
4. Just state the fact directly

Example:
Question: What is the person's email?
Bad: "The person's email address is john@example.com"
Good: "john@example.com"

Question: What is the person's name?
Bad: "According to the CV, the person's name is John Doe"
Good: "John Doe"

Now answer the actual question above with just the direct answer:"""

    print("🤖 Generating answer...\n")

    try:
        response = model.generate_content(prompt)
        answer = response.text.strip()
    except Exception as e:
        answer = f"Error: {str(e)}"

    # Display result
    print("="*60)
    print(f"❓ Question: {question}")
    print("="*60)
    print(f"💡 Answer: {answer}")
    print("="*60)

    return answer

print("✅ RAG function ready!")

✅ RAG function ready!


In [48]:
ask_cv_question("Give a short summary of CV?", k=10)

🔍 Searching for relevant info...
✅ Found 10 relevant sections

🤖 Generating answer...

❓ Question: Give a short summary of CV?
💡 Answer: Entry-Level AI/Machine Learning Engineer with a Bachelor of Science in Computer Science, experienced


'Entry-Level AI/Machine Learning Engineer with a Bachelor of Science in Computer Science, experienced'